In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln)
        )
        self.input_proj = nn.Sequential(nn.Linear(config.dim_input, config.dim_hidden), nn.ReLU())
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
        )
        self.regr = nn.Sequential(
            nn.ReLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_18_less_regression'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_18_less_regression-2024-11-22-17-43-54


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/120000 [00:00<?, ?it/s]

{'loss': 0.0214, 'grad_norm': 3.662355422973633, 'learning_rate': 4.999583333333333e-05, 'epoch': 0.0}
{'loss': 0.0103, 'grad_norm': 2.181835174560547, 'learning_rate': 4.999166666666667e-05, 'epoch': 0.0}
{'loss': 0.0081, 'grad_norm': 3.4937868118286133, 'learning_rate': 4.99875e-05, 'epoch': 0.0}
{'loss': 0.0091, 'grad_norm': 2.776449680328369, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.0086, 'grad_norm': 2.3532614707946777, 'learning_rate': 4.997916666666667e-05, 'epoch': 0.0}
{'loss': 0.0082, 'grad_norm': 2.372020721435547, 'learning_rate': 4.9975e-05, 'epoch': 0.0}
{'loss': 0.011, 'grad_norm': 2.00980281829834, 'learning_rate': 4.997083333333333e-05, 'epoch': 0.0}
{'loss': 0.01, 'grad_norm': 2.027223825454712, 'learning_rate': 4.996666666666667e-05, 'epoch': 0.01}
{'loss': 0.0085, 'grad_norm': 0.47187724709510803, 'learning_rate': 4.99625e-05, 'epoch': 0.01}
{'loss': 0.0075, 'grad_norm': 1.628836750984192, 'learning_rate': 4.995833333333333e-05, 'epoch': 0.01

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0028167578857392073, 'eval_runtime': 152.4989, 'eval_samples_per_second': 1311.485, 'eval_steps_per_second': 32.787, 'epoch': 1.0}
{'loss': 0.0037, 'grad_norm': 0.5281669497489929, 'learning_rate': 4.374583333333334e-05, 'epoch': 1.0}
{'loss': 0.0042, 'grad_norm': 0.5736628770828247, 'learning_rate': 4.374166666666667e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.6112236976623535, 'learning_rate': 4.3737500000000006e-05, 'epoch': 1.0}
{'loss': 0.0033, 'grad_norm': 1.1067897081375122, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.0033, 'grad_norm': 1.1679174900054932, 'learning_rate': 4.372916666666667e-05, 'epoch': 1.0}
{'loss': 0.0035, 'grad_norm': 1.4877249002456665, 'learning_rate': 4.3725000000000006e-05, 'epoch': 1.0}
{'loss': 0.0038, 'grad_norm': 1.2608072757720947, 'learning_rate': 4.372083333333333e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.5541022419929504, 'learning_rate': 4.371666666666667e-05, 'epoch': 1.01}
{'loss': 0.0038, '

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.002531040459871292, 'eval_runtime': 152.8802, 'eval_samples_per_second': 1308.214, 'eval_steps_per_second': 32.705, 'epoch': 2.0}
{'loss': 0.0027, 'grad_norm': 0.4085925817489624, 'learning_rate': 3.7495833333333334e-05, 'epoch': 2.0}
{'loss': 0.0021, 'grad_norm': 0.3746516704559326, 'learning_rate': 3.749166666666667e-05, 'epoch': 2.0}
{'loss': 0.0017, 'grad_norm': 0.6820699572563171, 'learning_rate': 3.74875e-05, 'epoch': 2.0}
{'loss': 0.002, 'grad_norm': 0.7973092794418335, 'learning_rate': 3.7483333333333334e-05, 'epoch': 2.0}
{'loss': 0.0018, 'grad_norm': 0.25542280077934265, 'learning_rate': 3.747916666666667e-05, 'epoch': 2.0}
{'loss': 0.0031, 'grad_norm': 0.5903359651565552, 'learning_rate': 3.7475e-05, 'epoch': 2.0}
{'loss': 0.0026, 'grad_norm': 0.6010993719100952, 'learning_rate': 3.7470833333333334e-05, 'epoch': 2.0}
{'loss': 0.0016, 'grad_norm': 0.326962947845459, 'learning_rate': 3.7466666666666665e-05, 'epoch': 2.01}
{'loss': 0.0032, 'grad_norm': 0.4080310

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0022683918941766024, 'eval_runtime': 150.8091, 'eval_samples_per_second': 1326.18, 'eval_steps_per_second': 33.154, 'epoch': 3.0}
{'loss': 0.0022, 'grad_norm': 0.5246175527572632, 'learning_rate': 3.124583333333334e-05, 'epoch': 3.0}
{'loss': 0.0025, 'grad_norm': 0.5975340008735657, 'learning_rate': 3.124166666666667e-05, 'epoch': 3.0}
{'loss': 0.0027, 'grad_norm': 0.3693516254425049, 'learning_rate': 3.12375e-05, 'epoch': 3.0}
{'loss': 0.0014, 'grad_norm': 0.7934311628341675, 'learning_rate': 3.123333333333334e-05, 'epoch': 3.0}
{'loss': 0.0018, 'grad_norm': 0.6316849589347839, 'learning_rate': 3.122916666666667e-05, 'epoch': 3.0}
{'loss': 0.0026, 'grad_norm': 0.8102643489837646, 'learning_rate': 3.122500000000001e-05, 'epoch': 3.0}
{'loss': 0.0038, 'grad_norm': 0.5260636806488037, 'learning_rate': 3.122083333333333e-05, 'epoch': 3.0}
{'loss': 0.0026, 'grad_norm': 0.16830161213874817, 'learning_rate': 3.121666666666667e-05, 'epoch': 3.01}
{'loss': 0.0027, 'grad_norm': 

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0019863848574459553, 'eval_runtime': 150.9588, 'eval_samples_per_second': 1324.865, 'eval_steps_per_second': 33.122, 'epoch': 4.0}
{'loss': 0.002, 'grad_norm': 0.28861716389656067, 'learning_rate': 2.4995833333333336e-05, 'epoch': 4.0}
{'loss': 0.0022, 'grad_norm': 0.34882381558418274, 'learning_rate': 2.499166666666667e-05, 'epoch': 4.0}
{'loss': 0.0022, 'grad_norm': 0.10788415372371674, 'learning_rate': 2.49875e-05, 'epoch': 4.0}
{'loss': 0.0025, 'grad_norm': 0.042367517948150635, 'learning_rate': 2.4983333333333335e-05, 'epoch': 4.0}
{'loss': 0.0015, 'grad_norm': 0.36160990595817566, 'learning_rate': 2.4979166666666666e-05, 'epoch': 4.0}
{'loss': 0.0021, 'grad_norm': 0.2549111247062683, 'learning_rate': 2.4975e-05, 'epoch': 4.0}
{'loss': 0.0015, 'grad_norm': 0.5441840887069702, 'learning_rate': 2.4970833333333335e-05, 'epoch': 4.0}
{'loss': 0.0018, 'grad_norm': 0.14642545580863953, 'learning_rate': 2.496666666666667e-05, 'epoch': 4.01}
{'loss': 0.0028, 'grad_norm': 0

In [ ]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [ ]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [ ]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]